In [13]:
import numpy as np
import data_managers
import network_optimizers
import lstm_network_components
import theano.tensor as T
import theano

In [2]:
# Hire a data butler
data_butler = data_managers.character_data_manager('/Users/alex/Desktop/data manager/', batch_size=10)

In [36]:
# Make a network for testing things

reload(lstm_network_components)

# LSTM stack details
stack_specs = [
    (128,100),
    (128,100)
    ]

# Network components
tst_stack = lstm_network_components.LSTM_stack( data_butler.ent_vecs.shape[1], stack_specs )
tst_stack.initialize_stack_weights()
tst_SR = lstm_network_components.soft_reader( sum([L.num_outputs for L in tst_stack.layers]), data_butler.ent_vecs.shape[0]-1 )

# Symbolic players
tst_inp         = T.tensor3()
tst_seq_lengths = T.ivector()
tst_targets     = T.dmatrix()

# Symbolic outputs
ys = tst_stack.process(tst_inp,tst_seq_lengths)
ps = tst_SR.process(ys)
cost = T.sum(T.nnet.binary_crossentropy(ps,tst_targets))

# Functions to run the graph with real variables
process_fun = theano.function([tst_inp,tst_seq_lengths], [ys,ps])
cost_fun    = theano.function([tst_inp,tst_seq_lengths,tst_targets],cost)

#Test that the network and data butler get along

In [40]:
data_butler.set_batch_size(10)

In [41]:
# Use this to get data and ensure proper training scheduling
data_butler.advance_schedule()
V,L,A = data_butler.offer_data()
L

[1170, 1415, 1283, 1201, 1177, 1567, 1282, 1290, 1566, 1577]

In [42]:
%%time
# Use this to ensure the network runs

Y,P = process_fun(V,L)
C = cost_fun(V,L,A)

CPU times: user 4.85 s, sys: 580 ms, total: 5.43 s
Wall time: 3.77 s


#Test training initialization

In [193]:
inp_list   = [tst_inp, tst_seq_lengths, tst_targets]
param_list = tst_stack.list_params() + tst_SR.list_params()

adam

In [83]:
%%time
train_adam = network_optimizers.adam_loves_theano(inp_list, cost, param_list)

CPU times: user 3min 14s, sys: 3.55 s, total: 3min 17s
Wall time: 3min 24s


adadelta

In [194]:
%%time
train_adadelta = network_optimizers.adadelta_fears_committment(inp_list, cost, param_list)

CPU times: user 3min 14s, sys: 6.78 s, total: 3min 21s
Wall time: 3min 27s


SGD

In [195]:
%%time
train_SGD = network_optimizers.i_hate_SGD(inp_list, cost, param_list)

CPU times: user 1min 31s, sys: 3.2 s, total: 1min 34s
Wall time: 1min 39s


#Make sure trianing works

In [196]:
data_butler.set_batch_size(3)

In [197]:
# This should work here because it worked above... right?
data_butler.advance_schedule()
V,L,A = data_butler.offer_data()
print len(L), max(L)

3 1191


adam

In [204]:
%%time
print 'Cost should go down:'
# Try a few training steps
print train_adam(V,L,A)
print train_adam(V,L,A)
print train_adam(V,L,A)

Cost should go down:
17.8032196945
17.6432914141
16.00736478
CPU times: user 36.6 s, sys: 7.05 s, total: 43.6 s
Wall time: 25.5 s


adadelta

In [205]:
%%time
print 'Cost should go down:'
# Try a few training steps
print train_adadelta(V,L,A)
print train_adadelta(V,L,A)
print train_adadelta(V,L,A)

Cost should go down:
17.4561301421
16.4719853629
7.70729775281
CPU times: user 36.5 s, sys: 7.1 s, total: 43.6 s
Wall time: 25.5 s


SGD

In [206]:
%%time
print 'Cost should go down (but maybe not because SGD sucks):'
# Try a few training steps
print train_SGD(V,L,A)
print train_SGD(V,L,A)
print train_SGD(V,L,A)

Cost should go down:
6.87551611886
19.4243090837
14.0978128436
CPU times: user 34.1 s, sys: 6.74 s, total: 40.8 s
Wall time: 23.7 s
